# ETL: Limpieza de Datos de Viviendas en Barcelona (PySpark)

## Importamos las librerías necesarias


In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re


# Crear SparkSession
spark = SparkSession.builder \
    .appName("ETL_Housing_Barcelona") \
    .getOrCreate()

print("✅ SparkSession creada correctamente")
print(f"Versión de Spark: {spark.version}")


✅ SparkSession creada correctamente
Versión de Spark: 4.0.1


## EXTRACT: Cargar los datos


In [59]:
# Cargar el CSV
df_raw = spark.read.csv(
    "../data/housing-barcelona.csv",
    header=True,
    inferSchema=False  # Leer todo como string primero
)

print("✅ El dataframe se ha creado correctamente")
print(f"Filas: {df_raw.count()}")
print(f"Columnas: {len(df_raw.columns)}")
print(f"\nColumnas: {df_raw.columns}")
print(f"\nPrimeras filas:")
df_raw.show(5, truncate=False)


✅ El dataframe se ha creado correctamente
Filas: 10000
Columnas: 20

Columnas: ['listing_id', 'operation', 'district', 'neighborhood', 'address', 'surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'floor', 'elevator', 'balcony', 'furnished', 'condition', 'energy_certificate', 'has_parking', 'latitude', 'longitude', 'agency']

Primeras filas:
+----------+---------+----------+---------------+-----------------+----------+-----+---------+---------+------------+------+--------+-------+---------+----------+------------------+-----------+--------+---------+---------------+
|listing_id|operation|district  |neighborhood   |address          |surface_m2|rooms|bathrooms|price_eur|price_per_m2|floor |elevator|balcony|furnished|condition |energy_certificate|has_parking|latitude|longitude|agency         |
+----------+---------+----------+---------------+-----------------+----------+-----+---------+---------+------------+------+--------+-------+---------+----------+------------------+---

## TRANSFORM: Limpieza de Datos

### Paso 1: Crear copia para trabajar


In [60]:
# Crear copia del dataframe (en PySpark, los DataFrames son inmutables, así que trabajamos directamente)
df_clean = df_raw
print(f"✅ Dataframe listo para trabajar. Filas: {df_clean.count()}")


✅ Dataframe listo para trabajar. Filas: 10000


### Paso 2: Eliminar espacios (strip) en columnas de texto


In [61]:
# Aplicar trim() a todas las columnas (elimina espacios al inicio y final)
for col_name in df_clean.columns:
    df_clean = df_clean.withColumn(col_name, trim(col(col_name)))

print("✅ Espacios eliminados de todas las columnas")


✅ Espacios eliminados de todas las columnas


### Paso 3: Reemplazar valores vacíos por NULL


In [62]:
# Reemplazar valores que representan "vacío" por NULL (None en PySpark)
valores_vacios = ['', ' ', 'nan', 'None', 'N/A', 'n/a', 'NULL', 'null', '?', 'unknown']

for col_name in df_clean.columns:
    df_clean = df_clean.withColumn(
        col_name,
        when(col(col_name).isin(valores_vacios) | col(col_name).isNull(), None)
        .otherwise(col(col_name))
    )

print("✅ Valores vacíos convertidos a NULL")
print(f"\nValores NULL por columna:")
null_counts = {}
for col_name in df_clean.columns:
    null_count = df_clean.filter(col(col_name).isNull()).count()
    if null_count > 0:
        null_counts[col_name] = null_count

for col_name, count in sorted(null_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {col_name}: {count}")


✅ Valores vacíos convertidos a NULL

Valores NULL por columna:
  latitude: 5055
  price_per_m2: 5041
  longitude: 4919
  rooms: 4004
  bathrooms: 3992
  surface_m2: 3983
  furnished: 3907
  address: 3904
  price_eur: 3347
  listing_id: 3321
  balcony: 2579
  condition: 2533
  elevator: 2451
  has_parking: 2447
  energy_certificate: 2234
  neighborhood: 1425
  agency: 1422
  operation: 1399
  floor: 1277


### Paso 4: Convertir tipos de datos adecuados

Primero definimos UDFs (User Defined Functions) para las funciones personalizadas


In [63]:
# Definir UDFs para extraer números y convertir texto

# UDF para extraer números de strings
def extract_number_udf(value):
    """Extrae el primer número de un string"""
    if value is None:
        return None
    value_str = str(value)
    numbers = re.findall(r'\d+\.?\d*', value_str)
    if numbers:
        return float(numbers[0])
    return None

# UDF para convertir texto a número
def text_to_number_udf(value):
    """Convierte texto a número"""
    if value is None:
        return None
    value_str = str(value).lower().strip()
    
    text_map = {
        'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
        'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10
    }
    
    if value_str in text_map:
        return text_map[value_str]
    
    if '+' in value_str:
        nums = re.findall(r'\d+', value_str)
        if nums:
            return int(nums[0])
    
    numbers = re.findall(r'\d+\.?\d*', value_str)
    if numbers:
        return float(numbers[0])
    return None

# UDF para extraer precio
def extract_price_udf(value):
    """Extrae precio numérico"""
    if value is None:
        return None
    value_str = str(value).replace('€', '').replace('.', '').replace(',', '.').strip()
    numbers = re.findall(r'\d+\.?\d*', value_str)
    if numbers:
        return float(numbers[0])
    return None

# UDF para extraer precio por m²
def extract_price_m2_udf(value):
    """Extrae precio por m²"""
    if value is None:
        return None
    value_str = str(value).replace('€/m2', '').replace('€/m²', '').replace('.', '').replace(',', '.').strip()
    numbers = re.findall(r'\d+\.?\d*', value_str)
    if numbers:
        return float(numbers[0])
    return None

# Registrar UDFs
extract_number = udf(extract_number_udf, DoubleType())
text_to_number = udf(text_to_number_udf, DoubleType())
extract_price = udf(extract_price_udf, DoubleType())
extract_price_m2 = udf(extract_price_m2_udf, DoubleType())

print("✅ UDFs definidas y registradas")


✅ UDFs definidas y registradas


In [64]:
# Limpiar columnas numéricas usando las UDFs

# Limpiar surface_m2
if 'surface_m2' in df_clean.columns:
    df_clean = df_clean.withColumn('surface_m2', extract_number('surface_m2'))

# Limpiar rooms
if 'rooms' in df_clean.columns:
    df_clean = df_clean.withColumn('rooms', text_to_number('rooms'))

# Limpiar bathrooms
if 'bathrooms' in df_clean.columns:
    df_clean = df_clean.withColumn('bathrooms', text_to_number('bathrooms'))

# Limpiar price_eur
if 'price_eur' in df_clean.columns:
    df_clean = df_clean.withColumn('price_eur', extract_price('price_eur'))

# Limpiar price_per_m2
if 'price_per_m2' in df_clean.columns:
    df_clean = df_clean.withColumn('price_per_m2', extract_price_m2('price_per_m2'))

# Convertir coordenadas a numérico
if 'latitude' in df_clean.columns:
    df_clean = df_clean.withColumn('latitude', col('latitude').cast('double'))
if 'longitude' in df_clean.columns:
    df_clean = df_clean.withColumn('longitude', col('longitude').cast('double'))

print("✅ Columnas numéricas limpiadas y convertidas")
print(f"\nTipos de datos numéricos:")
df_clean.select('surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'latitude', 'longitude').printSchema()


✅ Columnas numéricas limpiadas y convertidas

Tipos de datos numéricos:
root
 |-- surface_m2: double (nullable = true)
 |-- rooms: double (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- price_eur: double (nullable = true)
 |-- price_per_m2: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [65]:
# Convertir columnas que deben ser enteros
if 'rooms' in df_clean.columns:
    df_clean = df_clean.withColumn('rooms', col('rooms').cast('int'))
if 'bathrooms' in df_clean.columns:
    df_clean = df_clean.withColumn('bathrooms', col('bathrooms').cast('int'))

print("✅ Columnas convertidas a enteros")


✅ Columnas convertidas a enteros


In [66]:
# Asegurar que las columnas de texto sean string
text_cols = ['listing_id', 'operation', 'district', 'neighborhood', 'address', 
             'floor', 'condition', 'energy_certificate', 'agency']

for col_name in text_cols:
    if col_name in df_clean.columns:
        df_clean = df_clean.withColumn(
            col_name,
            when(col(col_name).isNull() | (col(col_name) == 'nan'), None)
            .otherwise(col(col_name).cast('string'))
        )

print("✅ Columnas de texto convertidas a string")


✅ Columnas de texto convertidas a string


In [67]:
# Convertir columnas booleanas
boolean_cols = ['elevator', 'balcony', 'furnished', 'has_parking']

for col_name in boolean_cols:
    if col_name in df_clean.columns:
        df_clean = df_clean.withColumn(
            col_name,
            when(lower(trim(col(col_name))).isin(['y', 'yes', 'sí', 'si', 's', '1', 'true']), True)
            .when(lower(trim(col(col_name))).isin(['n', 'no', '0', 'false']), False)
            .otherwise(None)
        )

print("✅ Columnas booleanas convertidas")


✅ Columnas booleanas convertidas


### Paso 5: Rellenar valores faltantes


In [68]:
# Rellenar valores faltantes
# Para columnas de texto: rellenar con "{nombre_columna} empty"
# Para columnas numéricas: rellenar con la media
# Para columnas booleanas: rellenar con False

import builtins  # Para usar round() de Python en lugar del de PySpark

print("=== RELLENANDO VALORES FALTANTES ===\n")

# Obtener esquema para identificar tipos
schema = df_clean.schema

# Crear diccionario para fillna
fill_dict = {}

# Rellenar columnas de texto (StringType)
for field in schema.fields:
    if isinstance(field.dataType, StringType):
        fill_dict[field.name] = f"{field.name} empty"
        print(f"✅ {field.name}: valores rellenados con '{fill_dict[field.name]}'")

# Rellenar columnas numéricas con la media
numeric_cols = ['surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'latitude', 'longitude']

for col_name in numeric_cols:
    if col_name in df_clean.columns:
        # Calcular media
        mean_value = df_clean.agg(avg(col(col_name)).alias('mean')).collect()[0]['mean']
        
        if mean_value is not None:
            # Si es int, redondear usando round() de Python (builtins)
            if col_name in ['rooms', 'bathrooms']:
                mean_value = int(builtins.round(mean_value))
                fill_dict[col_name] = mean_value
                print(f"✅ {col_name}: valores rellenados con media = {mean_value} (entero)")
            else:
                fill_dict[col_name] = mean_value
                print(f"✅ {col_name}: valores rellenados con media = {mean_value:.2f}")

# Rellenar columnas booleanas con False
for field in schema.fields:
    if isinstance(field.dataType, BooleanType):
        fill_dict[field.name] = False
        print(f"✅ {field.name}: valores rellenados con False")

# Aplicar fillna
df_clean = df_clean.fillna(fill_dict)

# Verificar valores NULL restantes
null_count = 0
for col_name in df_clean.columns:
    col_null_count = df_clean.filter(col(col_name).isNull()).count()
    if col_null_count > 0:
        null_count += col_null_count

print(f"\n✅ Todos los valores faltantes han sido rellenados")
print(f"Valores NULL restantes: {null_count}")


=== RELLENANDO VALORES FALTANTES ===

✅ listing_id: valores rellenados con 'listing_id empty'
✅ operation: valores rellenados con 'operation empty'
✅ district: valores rellenados con 'district empty'
✅ neighborhood: valores rellenados con 'neighborhood empty'
✅ address: valores rellenados con 'address empty'
✅ floor: valores rellenados con 'floor empty'
✅ condition: valores rellenados con 'condition empty'
✅ energy_certificate: valores rellenados con 'energy_certificate empty'
✅ agency: valores rellenados con 'agency empty'
✅ surface_m2: valores rellenados con media = 106.58
✅ rooms: valores rellenados con media = 3 (entero)
✅ bathrooms: valores rellenados con media = 1 (entero)
✅ price_eur: valores rellenados con media = 263348.96
✅ price_per_m2: valores rellenados con media = 281963.15
✅ latitude: valores rellenados con media = 41.19
✅ longitude: valores rellenados con media = 2.08
✅ elevator: valores rellenados con False
✅ balcony: valores rellenados con False
✅ furnished: valores r

### Paso 6: Capitalizar texto (Title Case)


In [69]:
# Capitalizar texto en columnas de string (Title Case)
# UDF para capitalizar texto
def capitalize_text_udf(value):
    """Capitaliza la primera letra de cada palabra"""
    if value is None:
        return None
    # Convertir a string y capitalizar cada palabra
    return str(value).title()

capitalize_text = udf(capitalize_text_udf, StringType())

# Aplicar capitalización a columnas de texto (excepto las que tienen valores "X empty")
text_cols_to_capitalize = ['operation', 'district', 'neighborhood', 'condition', 'energy_certificate']

for col_name in text_cols_to_capitalize:
    if col_name in df_clean.columns:
        # Solo capitalizar si no es un valor "empty"
        df_clean = df_clean.withColumn(
            col_name,
            when(col(col_name).contains(" empty"), col(col_name))
            .otherwise(initcap(col(col_name)))  # initcap capitaliza la primera letra de cada palabra
        )

print("✅ Texto capitalizado (Title Case)")


✅ Texto capitalizado (Title Case)


### Verificación: Comparación antes/después


In [70]:
# Mostrar ejemplos de limpieza
print("=== EJEMPLOS DE LIMPIEZA ===\n")
print("ANTES (RAW):")
df_raw.select('surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'elevator', 'district').show(10, truncate=False)
print("\nDESPUÉS (CLEAN):")
df_clean.select('surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'elevator', 'district').show(10, truncate=False)


=== EJEMPLOS DE LIMPIEZA ===

ANTES (RAW):
+----------+-----+---------+---------+------------+--------+-------------------+
|surface_m2|rooms|bathrooms|price_eur|price_per_m2|elevator|district           |
+----------+-----+---------+---------+------------+--------+-------------------+
|89 m²     |?    |2        |?        |4240 €/m2   |Y       |Unknown            |
|171       |N/A  |1        |?        |7920.91     |?       |Eixampl            |
|?         |2+   |?        |317642 € |?           |Y       |Sant Martí         |
|N/A       |three|two      |N/A      |5484 €/m2   |N       |SANTS              |
|?         |2+   |?        |N/A      |?           |Sí      |SANTS              |
|127 m²    |three|2        |491626 € |N/A         |Y       |Ciutat Vella       |
|?         |2+   |two      |N/A      |?           |N       |Sants-Montjuïc     |
|?         |three|?        |1282371  |4093 €/m2   |Y       |Sarrià-Sant Gervasi|
|127 m²    |2+   |3        |?        |6630.1      |unknown |Les Co

### Resumen de la transformación


In [71]:
print("=== RESUMEN DE LA TRANSFORMACIÓN ===\n")
print(f"Filas: {df_clean.count()}")
print(f"Columnas: {len(df_clean.columns)}")
print(f"\nEsquema del dataset limpio:")
df_clean.printSchema()
print(f"\nPrimeras filas del dataset limpio:")
df_clean.show(5, truncate=False)


=== RESUMEN DE LA TRANSFORMACIÓN ===

Filas: 10000
Columnas: 20

Esquema del dataset limpio:
root
 |-- listing_id: string (nullable = false)
 |-- operation: string (nullable = false)
 |-- district: string (nullable = false)
 |-- neighborhood: string (nullable = false)
 |-- address: string (nullable = false)
 |-- surface_m2: double (nullable = false)
 |-- rooms: integer (nullable = false)
 |-- bathrooms: integer (nullable = false)
 |-- price_eur: double (nullable = false)
 |-- price_per_m2: double (nullable = false)
 |-- floor: string (nullable = false)
 |-- elevator: boolean (nullable = false)
 |-- balcony: boolean (nullable = false)
 |-- furnished: boolean (nullable = false)
 |-- condition: string (nullable = false)
 |-- energy_certificate: string (nullable = false)
 |-- has_parking: boolean (nullable = false)
 |-- latitude: double (nullable = false)
 |-- longitude: double (nullable = false)
 |-- agency: string (nullable = false)


Primeras filas del dataset limpio:
+----------------+

## LOAD: Guardar datos limpios


In [72]:
# Guardar el dataframe limpio como un solo archivo CSV
import os
import glob
import pandas as pd
import shutil

output_path = "../data/housing-barcelona-clean-pyspark.csv"
temp_dir = "../data/temp_pyspark_output"

# Eliminar el directorio de salida si existe (por si hay una ejecución anterior)
if os.path.exists(output_path):
    if os.path.isdir(output_path):
        shutil.rmtree(output_path)
    else:
        os.remove(output_path)

# Eliminar directorio temporal si existe
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)

# Guardar en directorio temporal con una sola partición
df_clean.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_dir)

# Encontrar el archivo CSV generado (PySpark crea archivos como part-00000-*.csv)
csv_files = glob.glob(os.path.join(temp_dir, "part-*.csv"))

if csv_files:
    # Leer el archivo CSV generado
    df_pandas = pd.read_csv(csv_files[0])
    
    # Guardar como un solo archivo CSV
    df_pandas.to_csv(output_path, index=False)
    
    # Eliminar el directorio temporal y su contenido
    shutil.rmtree(temp_dir)
    
    print(f"✅ Datos limpios guardados en: {output_path}")
    print(f"\nArchivo guardado exitosamente con {len(df_pandas)} filas y {len(df_pandas.columns)} columnas")
else:
    print("❌ Error: No se encontró el archivo CSV generado")


✅ Datos limpios guardados en: ../data/housing-barcelona-clean-pyspark.csv

Archivo guardado exitosamente con 10000 filas y 20 columnas
